# **3: DuckDB's Best Practices**

---

By Jean-Yves Tran | jy.tran@[datascience-jy.com](https://datascience-jy.com) | [LinkedIn](https://www.linkedin.com/in/jytran-datascience/)  
IBM Certified Data Analyst 

---

Source: 
- [Getting Started with DuckDB](https://www.packtpub.com/en-ar/product/getting-started-with-duckdb-9781803232539) by Simon Aubury & Ned Letcher
- [DuckDB documentation](https://duckdb.org/docs/)
---

The interactive links in this notebook are not working due to GitHub limitations. View this notebook with the interactive links working [here](https://nbviewer.org/github/jendives2000/Data_ML_Practice_2025/blob/main/1-3-SQL/practice/DuckDB/notebooks/2_duckdb_python_API.ipynb).

---

This is part 3 of the series of notebooks on DuckDB.  
In the previous two notebooks I've introduced DuckDB and shoed how to load, transform and briefly analyse data from different sources.  
Here I will learn about best practices that:
- save time when:
  - querying or inserting data to a DuckDB database
  - joining tables (positional and temporal joins)

For an introduction to DuckDB, check [my first notebook](https://github.com/jendives2000/Data_ML_Practice_2025/blob/82571ad44176666f9cf0735c5141c6a96d5eace9/1-3-SQL/practice/DuckDB/notebooks/1_duckdb_intro.ipynb). I also say in there when you should not use DuckDB. 

For understanding how DuckDB works, check my [second notebook](https://github.com/jendives2000/Data_ML_Practice_2025/blob/ef8533ad82586234cfdc54a494c0c5be590816cc/1-3-SQL/practice/DuckDB/notebooks/2_duckdb_python_API.ipynb).

In more details, I will cover the followings:
- **Selecting columns** effectively
- Applying **function chaining**
- Using **INSERT** effectively
- Leveraging **positional** joins and **temporal joins**
- **Recursive** queries and macros
- additional **tips and tricks**

**SKIERS DATABASE**:  
I added a very simple Skiers Database in the data/data_in folder: `skiers.csv`  
It'll be used throughout this notebook. 

**The two main takeaways are**:


---


## Imports:

In [ ]:
# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))
from utils.duckdb_shared_code import

## Table setup using the DuckDB Shell:

I want to work with the DuckDB shell, from my Jupyter notebook. It is possible by using the following syntax:

In [11]:
!duckdb mydatabase.duckdb -c "create or replace table skiers as select * from read_csv('../data/data_in/skiers.csv');"

In [13]:
!duckdb mydatabase.duckdb -c "show tables"

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ skiers  │
└─────────┘


In [23]:
query = "select * from skiers;" 
shell_commd(query) 

┌──────────────────┬─────────────────┬───────────┬──────────────┬────────────────────┬─────────────────┐
│ skier_first_name │ skier_last_name │ skier_age │ skier_height │ skier_helmet_color │ skier_bib_color │
│     varchar      │     varchar     │   int64   │    int64     │      varchar       │     varchar     │
├──────────────────┼─────────────────┼───────────┼──────────────┼────────────────────┼─────────────────┤
│ Alice            │ Smith           │        12 │          152 │ red                │ black           │
│ Bob              │ Blaese          │        16 │          178 │ blue               │ yellow          │
│ Carol            │ Wilson          │        32 │          159 │ yellow             │ pink            │
│ Dan              │ Jones           │        52 │          182 │ red                │ yellow          │
│ Erin             │ Taylor          │        22 │          168 │ black              │ green           │
│ Frank            │ Williams        │        18 │     

Let's refactor this syntax and copy it over to the `duckdb_shared_code.py` file. 

In [22]:
def shell_commd(stmt: str):
    get_ipython().system(f"duckdb mydatabase.duckdb -c \"{stmt}\"")

I want to see the whole table skiers now:

In [27]:
shell_commd(
    "select * from skiers;"
)

┌──────────────────┬─────────────────┬───────────┬──────────────┬────────────────────┬─────────────────┐
│ skier_first_name │ skier_last_name │ skier_age │ skier_height │ skier_helmet_color │ skier_bib_color │
│     varchar      │     varchar     │   int64   │    int64     │      varchar       │     varchar     │
├──────────────────┼─────────────────┼───────────┼──────────────┼────────────────────┼─────────────────┤
│ Alice            │ Smith           │        12 │          152 │ red                │ black           │
│ Bob              │ Blaese          │        16 │          178 │ blue               │ yellow          │
│ Carol            │ Wilson          │        32 │          159 │ yellow             │ pink            │
│ Dan              │ Jones           │        52 │          182 │ red                │ yellow          │
│ Erin             │ Taylor          │        22 │          168 │ black              │ green           │
│ Frank            │ Williams        │        18 │     